In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import dailydata
import requests
import collections

from sklearn import linear_model
import statsmodels.api as sm

from backtesting.test import GOOG

/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
with open("words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]
# words = words[:5] + ["debt"]
# words = ["debt"]
#words = words[:10]

word_trends = {}

In [3]:
def getStats(key_word, start_year, start_month, end_year, end_month, index):
    trend = pd.DataFrame(dailydata.get_daily_data(key_word, start_year, start_month, end_year, end_month, wait_time = 0)[key_word])
    trend["change"] = trend.pct_change()

    # grab DJIA data
    index_data = yf.download(index, start = str(start_year)+"-"+str(start_month)+"-01", end = str(end_year)+"-"+str(end_month)+"-01")

    # join trend data with DJIA data
    joined = trend.merge(index_data, left_on = trend.index, right_on = index_data.index)
    joined = joined.rename(columns = {"key_0": "Date"})

    # grab the adj close price difference for each day
    #joined['Diff'] = joined['Adj Close'].diff()
    joined = joined.set_index(joined['Date'])

    joined["change moving avg"] = joined["change"].rolling("7d", min_periods = 1).mean()
    joined = joined[joined.index.dayofweek == 1]
    
    return joined




In [4]:
start_year = 2014
start_month = 1
end_year = 2014
end_month = 12
index = "DJIA"

for word in words:
    word_trends[word] = getStats(word, start_year, start_month, end_year, end_month, index)

The request failed: Google returned a response with code 429.
Trying again in 60 seconds.
The request failed: Google returned a response with code 429.
Trying again in 65 seconds.
The request failed: Google returned a response with code 429.
Trying again in 70 seconds.
The request failed: Google returned a response with code 429.
Trying again in 75 seconds.
Failed after 3 attemps, abort fetching.


KeyError: 'request'

In [ ]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [ ]:
def ourFunc(df):
    return pd.Series(df['change'])

def ourFunc1(df):
    return pd.Series(df['change moving avg'])

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover


class ourStrat(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    #n1 = 10
    #n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        #self.sma1 = self.I(SMA, self.data.Close, self.n1)
        #self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.change = self.I(ourFunc1, self.data)
        self.own = False
        #self.buy_from_last_week = False
        #self.sell_from_last_week = True
    
    def next(self):
        """
        if self.buy_from_last_week:
            self.buy()
            self.own = True
            self.buy_from_last_week = False
        
        elif self.sell_from_last_week:
            self.sell()
            self.own = False
            self.sell_from_last_week = False
        """
        
        if self.change[-1] > 0: #and not self.own: #crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
            #self.own = True

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif self.change[-1] < 0: #and self.own: #crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            #self.own = False

In [ ]:
from backtesting import Backtest

bt = Backtest(joined, ourStrat, cash=100_000_000, commission=.002)
stats = bt.run()
stats

In [ ]:
returns = {}

for word in word_trends.keys():
    bt = Backtest(word_trends[word], ourStrat, cash=100_000_000, commission=0)
    stats = bt.run()
    returns[word] = stats[6]
    returns['BENCHMARK'] = stats[7]

returns

In [ ]:
word_trends['economics'].head()

In [ ]:
word_trends['invest'].head()